In [252]:
import pandas as pd 
import matplotlib.pyplot as plt 
import networkx as nx 



In [253]:
raw_data = pd.read_csv("raw_data.csv", header=None)
ignore_subject = set([1736438, 1740312, 1718449, 1716494, 1730150, 1720400, 1710201, 1710300, 1710404, 1712301, 1712404, 1714304, 1714406,1714429, 1714469, 1716401, 1716440, 1716444, 1716452, 1718404, 1718406, 1718452, 1720415, 1730101, 1730108, 1730404, 1732204])
all_subject = set(raw_data.iloc[:, 3])
our_subject = sorted(list(all_subject - ignore_subject))
print(our_subject)

[1710105, 1710204, 1710210, 1710217, 1710250, 1710417, 1712212, 1712312, 1712315, 1712401, 1712417, 1712432, 1712444, 1712452, 1714297, 1714303, 1714308, 1714320, 1714396, 1714417, 1714480, 1716310, 1716405, 1716408, 1716410, 1716430, 1716448, 1718210, 1718217, 1718303, 1718312, 1718402, 1718416, 1718417, 1718425, 1718436, 1718448, 1718450, 1720405, 1730115, 1730120, 1730151, 1730155, 1730217, 1730220, 1730400, 1730402, 1730403, 1730420, 1730430, 1730435, 1730460, 1730499, 1732203, 1732207, 1732208, 1732303, 1732308, 1732312, 1732401, 1732417, 1734102, 1734212, 1734303, 1734308, 1734333, 1734400, 1734425, 1734449, 1734452, 1736310, 1740320, 1740350, 1740404, 1740417]


In [254]:
def CrossCount(first, second):
    mask1 = raw_data.iloc[:,3] == first
    mask2 = raw_data.iloc[:,4] == second
    mask = mask2 & mask1

    group = raw_data[mask]
    return sum(group.iloc[:, 2])

In [255]:
cross_matrix = [[0 for j in our_subject] for i in our_subject]

for x in range(len(our_subject)):
    for y in range(len(our_subject)):
        cross_matrix[x][y] = max(CrossCount(our_subject[x], our_subject[y]), CrossCount(our_subject[y], our_subject[x]))

In [256]:
cross_matrix_df = pd.DataFrame(cross_matrix, index=our_subject, columns=our_subject)
print(cross_matrix_df)

         1710105  1710204  1710210  1710217  1710250  1710417  1712212   
1710105      200      157        8       21        0        0       12  \
1710204      157      296        6       11        0        0        8   
1710210        8        6      105        0        0        0       98   
1710217       21       11        0       63        0        0        0   
1710250        0        0        0        0       73        0        0   
...          ...      ...      ...      ...      ...      ...      ...   
1736310        0        0        0        0        0        0        0   
1740320        0        9        0        0        0        0        0   
1740350        1        2        0        0        0        8        0   
1740404        0        1        0        0        0        0        0   
1740417        2        0        0        0        0        0        1   

         1712312  1712315  1712401  ...  1734333  1734400  1734425  1734449   
1710105        2        1       

In [257]:
import os 
os.remove("cross_matrix.csv")
cross_matrix_df.to_csv("cross_matrix.csv")

In [258]:
edges = []
for i in range(len(our_subject)):
  for j in range(len(our_subject)):
   if cross_matrix[i][j] == 0: 
      edges.append([i,j])

In [259]:
G = nx.Graph()
G.add_edges_from(edges)
cliques = list(nx.find_cliques(G))
all_kernel_number = []
all_kernel_code = []

cliques = sorted(cliques, key=lambda x: len(x), reverse=True)
print(len(cliques))
print("All complete subgraphs (cliques):")
for clique in cliques:
    all_kernel_code.append([our_subject[i] for i in clique])
    all_kernel_number.append(set(clique))


37898
All complete subgraphs (cliques):


In [260]:
def greedy_set_cover(universe, subsets):
    covered = set()  # Tracks covered elements
    cover_sets = []  # Holds chosen subsets
    
    counter = 0
    while covered != universe:
        counter += 1
        if counter > len(subsets)**2:
            print("Infinite loop detected.")
            break
        best_subset = None
        max_covered = set()
        
        for subset in subsets:
            new_covered = subset - covered  # Calculate elements covered by the subset
            if len(new_covered) > len(max_covered):
                best_subset = subset
                max_covered = new_covered
        
        if(best_subset == None):
            break
        cover_sets.append(best_subset)
        covered |= max_covered
    
    return cover_sets

universe = set()
for subset in all_kernel_number:
    universe |= subset
print("Universe:", universe)


import random 
for i in range(1000):
    random.shuffle(all_kernel_number)
    minimum_cover = greedy_set_cover(universe, all_kernel_number)
    if len(minimum_cover) < 16:
        print("Minimum sets for cover:", minimum_cover)
        print(len(minimum_cover))

Universe: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74}


In [261]:
x = [len(i) for i in all_kernel_number]

unique_x = list(set(x))
for i in unique_x:
    print(i, x.count(i))


4 4
5 46
6 304
7 1310
8 4102
9 8684
10 11146
11 8358
12 3484
13 452
14 8
